# 1. Import Packages 

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, MaxPooling2D, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

tfd = tfp.distributions
tfpl = tfp.layers

# 2. Load and Generate Image

In [ ]:
labels = pd.read_csv('./archive/trainLabels_cropped.csv')
root_dir=os.path.join('./levels')
batch_size=64
img_width=256
img_height=256

def image_generator(root_dir):
    
    datagen = ImageDataGenerator(rescale=1/255,
                                      #shear_range=0.2,
                                      #zoom_range=0.2,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      validation_split=0.2)
  
    train_generator = datagen.flow_from_directory(root_dir,
                                  target_size = (img_width, img_height),
                                  batch_size = batch_size,
                                  class_mode = 'categorical',
                                  subset='training')
 
    
    test_generator = datagen.flow_from_directory(root_dir,
                                 target_size=(img_width, img_height),
                                 batch_size = batch_size,
                                 class_mode = 'categorical',
                                 subset='validation')    
    
    return train_generator, test_generator

train_generator, test_generator = image_generator(root_dir)
divergence_fn = lambda q,p,_:tfd.kl_divergence(q,p)/train_generator.samples

# 3. Bayesian by backprop Moedels 
+ In this section we introduced 3 bayesian models 
    + Bayesian Model 1 is a Bayesian by back prop model usign the repramaterization trick The repramatrization trick is an approximative way in sloving bayesian function 
    + Bayesian Model2 is another bayesian by backprop mehtods that repalces the repramaterization trick/ repramtrization layers with the flipout layers. The flipout layers use Montecarol approximation to solve the bayesian function. 
        + one advanteage of that method is its faster training time and more weights/ more degrees of freedom.
        
  + Bayesain Model 3 is an imporvment over bayesian modle 2 withere the trainign proces was
  

## Bayesian Model 1 

In [ ]:
model_bayes = Sequential([    
    tfpl.Convolution2DReparameterization(input_shape=(img_width, img_height,3),padding="same",filters=8, kernel_size=16, activation='relu',
                                           kernel_prior_fn = tfpl.default_multivariate_normal_fn,
                                           kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           kernel_divergence_fn = divergence_fn,
                                           bias_prior_fn = tfpl.default_multivariate_normal_fn,
                                           bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           bias_divergence_fn = divergence_fn),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    Conv2D(256, (3,3), activation='relu'),
    Conv2D(256, (3,3), activation='relu'),
    Conv2D(256, (3,3), activation='relu'),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    tfpl.DenseReparameterization(units=tfpl.OneHotCategorical.params_size(5), activation=None,
                                    kernel_prior_fn = tfpl.default_multivariate_normal_fn,
                                    kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                    kernel_divergence_fn = divergence_fn,
                                    bias_prior_fn = tfpl.default_multivariate_normal_fn,
                                    bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                    bias_divergence_fn = divergence_fn
                                ),
    tfpl.OneHotCategorical(5)
    
])
model_bayes.summary()

learning_rate = 0.005
def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

## Bayesian Model 2

In [ ]:
model_bayes = tf.keras.Sequential([
    tf.keras.Input(shape=(img_width, img_height, 3),name="basket"),
    
    tfpl.Convolution2DFlipout(16, kernel_size=5, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1a", 
                                    kernel_divergence_fn=divergence_fn),

    MaxPool2D(strides=(4,4), pool_size=(4,4), padding="same"),
    Conv2D(32, (3,3), activation='relu'),
    
    tfpl.Convolution2DFlipout(32, kernel_size=3, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1b", 
                                    kernel_divergence_fn=divergence_fn),
    MaxPool2D(strides=(4,4), pool_size=(4,4), padding="same"),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    
    Dropout(0.2),
    tfpl.DenseFlipout(units=5, kernel_divergence_fn=divergence_fn),
    #tfpl.OneHotCategorical(5)
])
model_bayes.summary()

learning_rate = 1.0e-3
def negative_log_likelihood(y_true, y_pred):
    y_pred = tfp.distributions.MultivariateNormalTriL(y_pred)
    return -tf.reduce_mean(y_pred.log_prob(y_true))

## Bayesian Model 3

In [ ]:
model_bayes = tf.keras.Sequential([
    tf.keras.Input(shape=(img_width, img_height, 3),name="basket"),
    
    tfpl.Convolution2DFlipout(16, kernel_size=5, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1a", 
                                    kernel_divergence_fn=divergence_fn),
    MaxPool2D(strides=(2,2), pool_size=(2,2), padding="same"),
    Conv2D(128, (3,3), activation='relu'),
    MaxPool2D(strides=(2,2), pool_size=(2,2), padding="same"),
    tfpl.Convolution2DFlipout(32, kernel_size=3, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1b", 
                                    kernel_divergence_fn=divergence_fn),
    MaxPool2D(strides=(2,2), pool_size=(2,2), padding="same"),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    
    Dropout(0.2),
    tfpl.DenseFlipout(units=5,activation='softmax', kernel_divergence_fn=divergence_fn),
    #tfpl.OneHotCategorical(5)
])
model_bayes.summary()
learning_rate = 1.0e-3

def negative_log_likelihood(y_true, y_pred):
    y_pred = tfp.distributions.MultivariateNormalTriL(y_pred)
    return -tf.reduce_mean(y_pred.log_prob(y_true))

# 4. Compile and Train the Model 

In [ ]:
def class0(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, 0), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

def class1(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, 1), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

def class2(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, 2), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

def class3(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, 3), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

def class4(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, 4), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

model_bayes.compile(loss = negative_log_likelihood,
              optimizer = Adam(learning_rate=learning_rate),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),class0,class1,class2,class3,class4],
              experimental_run_tf_function = False)

In [ ]:
history = model_bayes.fit(train_generator,
                                steps_per_epoch = train_generator.samples // batch_size,
                                validation_data = test_generator, 
                                validation_steps = test_generator.samples // batch_size,
                                epochs = 100)

# 5. Data Analysis

In [ ]:
np.savez("history_plots.npz",history.history)

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12, 3))
fig.suptitle("Bayesian Model 2's accuracy and Loss",fontsize=12)


ax1.plot(history2.history['CategoricalAccuracy'],label='train')
ax1.plot(history2.history['val_CategoricalAccuracy'],label='test')
ax1.title.set_text('model accuracy')
ax1.set_ylabel('accuracy')
ax1.set_xlabel('epoch')
ax1.legend()

ax2.plot(history2.history['loss'],label='train')
ax2.plot(history2.history['val_loss'],label='test')
ax2.title.set_text('model loss')
ax2.set_ylabel('loss')
ax2.set_xlabel('epoch')
ax2.legend()

In [ ]:
def import_and_predict_bayes(image, true_label):
    #read image
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    #show the image
    plt.imshow(img)
    plt.axis('off')
    img_resize = (cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC))/255.
    predicted_probabilities = np.empty(shape=(300, 5))
    predicted_probabilities2 = np.empty(shape=(300, 5))
    predicted_probabilities3 = np.empty(shape=(300, 5))
    for i in range(300):
        predicted_probabilities2[i] =model_bayes3.predict(img_resize[np.newaxis,...])   
    pct_2p5 = np.array([np.percentile(predicted_probabilities2[:, i], 2.5) for i in range(5)])
    pct_97p5 = np.array([np.percentile(predicted_probabilities2[:, i], 97.5) for i in range(5)])
    fig, ax = plt.subplots(figsize=(12, 6))
    bar = ax.bar(np.arange(5), pct_97p5-0.02, color='red')
    bar[true_label].set_color('green')
    bar = ax.bar(np.arange(5), pct_2p5-0.02, color='white')
    ax.set_xticklabels([''] + [x for x in label],fontsize=20)
    ax.set_ylim([0, 1])
    ax.set_ylabel('Probability',fontsize=16)
    plt.show()
    
def import_and_predict2(image_data, label, T):    
    #read image
    img = cv2.imread(image_data)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    #show the image
    #plt.imshow(img)
    #plt.axis('off')
    # resize and reshape the image
    img_resize = (cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC))/255.
    img_reshape = img_resize[np.newaxis,...]
    #predict the image
    prediction = model_bayes3.predict(img_reshape)    
    p_hat=[]
    for t in range (T):
        p_hat.append(model_bayes3.predict(img_reshape)[0])
    p_hat=np.array(p_hat)
    prediction = np.mean(p_hat, axis=0)
    label_prediction = label[np.argmax(prediction)]
    aleatoric = np.mean(p_hat*(1-p_hat), axis=0)
    epistemic = np.mean(p_hat**2, axis=0) - np.mean(p_hat, axis=0)**2
    label_prediction = [np.argmax(prediction)]
    return prediction, label_prediction, aleatoric, epistemic
     

In [ ]:
label = os.listdir(root_dir)
image0_dir = os.path.join(root_dir+'/0/20_right.jpeg')
image0_dir2 = os.path.join(root_dir+'/0/70_left.jpeg')

image1_dir = os.path.join(root_dir+'/1/2161_left.jpeg')
image2_dir = os.path.join(root_dir+'/2/1034_left.jpeg')
image3_dir = os.path.join(root_dir+'/3/13489_left.jpeg')
image4_dir = os.path.join(root_dir+'/4/5032_right.jpeg')
print(root_dir)

prediction = import_and_predict_bayes(image0_dir2,  label.index('0'))

In [ ]:
input_dir=root_dir
Class_Names=["0","1","2","3","4"]
total_prediction=np.empty(5)
total_label_prediction=np.empty(1)
total_aleatoric=np.empty(5)
total_epistemic=np.empty(5)
total_accuracy=np.empty(1)
total_tracker=np.empty(1)
tracker=np.empty(1)

for Class in Class_Names:
    path = os.path.join(input_dir,Class)
    files = os.listdir(path)
    for file in files:
        T=10 # T should be 2 or greater
        
        image_dir=os.path.join(path,file)
        #print(image_dir)
        prediction, label_prediction, aleatoric, epistemic=import_and_predict2(image_dir, label,T)                        
        total_prediction=np.vstack((total_prediction,prediction))
        total_label_prediction=np.vstack((total_label_prediction,label_prediction))
        total_aleatoric=np.vstack((total_aleatoric,aleatoric))
        total_epistemic=np.vstack((total_epistemic,epistemic))
        tracker=Class_Names.index(Class)
        total_tracker=np.vstack((total_tracker,tracker))
                            
        if(int("".join(map(str, label_prediction)))==Class_Names.index(Class)):
                total_accuracy=np.vstack((total_accuracy,1))
        else: 
                total_accuracy=np.vstack((total_accuracy,0))

total_prediction1=total_prediction[1:,:]
total_label_prediction1=total_label_prediction[1:,:]
total_aleatoric1=total_aleatoric[1:,:]
total_epistemic1=total_epistemic[1:,:]
total_accuracy1=total_accuracy[1:,:]
total_tracker1=total_tracker[1:,:]                
np.savez("dataDR_bayes3_700.npz",total_prediction1,total_label_prediction1,total_aleatoric1,total_epistemic1,total_accuracy1,total_tracker1)